In [1]:
import pandas as pd
import numpy as np
import csv
import pickle
from collections import Counter
import matplotlib.pyplot as plt
import powerlaw,math
import os
import scipy.stats as stats
import random
from numpy import polyfit, poly1d

### 导出回归数据 (export regression data)

In [73]:
neuro = pd.read_csv('chemistry_0123.csv')

In [74]:
len(neuro)

25138

增加一列是否可以进入top5% (column : top5% or not)

In [77]:
neuro = neuro[(neuro.StopYear<=1995)&(neuro.StopYear>=1960)]

In [78]:
tp = neuro.StopYear
print(min(tp))
print(max(tp))

1960
1995


In [79]:
interval = [[1960,1970],[1970,1980],[1980,1990],[1985,1995]]
Top = {}
for itl in interval:
    i0 = itl[0];i1 = itl[1]
    temp = np.array(neuro[['CID','YearlyCitationOfMentee']][(neuro.StopYear>=i0)&(neuro.StopYear<=i1)])
    CID = temp[:,0]
    YC = temp[:,1]
    mm = sorted(temp,key = lambda x:x[1],reverse=1)
    
    pt = 0.05   #进入 top 5%
    th = mm[round(len(mm)*pt)][1]
    print(th)
    for i in range(len(CID)):
        if YC[i]>=th:
            Top[CID[i]]=1
        else:
            Top[CID[i]]=0

702.8833333333333
873.8888888888888
946.5121951219513
859.5641025641024


In [80]:
CID = neuro['CID']
b_or_s = []
for k in CID:
    b_or_s.append(Top[k])
neuro['Top25'] = b_or_s

In [81]:
neuro.shape

(18404, 32)

In [82]:
neuro_re = neuro[['MenteeID', 'MentorID','StopYear','BirthYearOfMentee','First5YearPubsOfMentee',
                  'First5YearCitationOfMentee','First5YearCollaOfMentee','CollaPubsOfWithMentor',
                  'BigOrSmall',
                  'TotalPubsOfMentor','TotalCitationOfMentor','TotalCollaOfMentor','CareerAgeOfMentorInTraining',
                  'YearlyPubsOfMentor','YearlyCitationOfMentor','YearllyCollaOfMentor','PubsOfMentorInTraning',
        
                  'Survival0','Survival1','YearlyCitationOfMentee','Top25','NumOfMenteesOfTheMenteeIn20']][(neuro.CareerAgeOfMentorInTraining<50)&(neuro.CareerAgeOfMentorInTraining>0)
                                             &(neuro.BigOrSmall!=-1)]
#neuro_era.BigOrSmall!=2)] #(neuro_era.TotalPubsOfMentor>0)&(neuro_era.CareerAgeOfMentorInTraining>-1)
                                    #(neuro_era.CareerAgeOfMentorInTraining<50)

In [83]:
neuro_re.shape

(9193, 22)

In [84]:
# neuro_re = neuro_re[(neuro_re.StopYear<=1995)&(neuro_re.StopYear>=1970)]

In [85]:
#测试大小团队中的生存率 (test survival rate)
all_big = neuro_re['MenteeID'][neuro_re.BigOrSmall==1]
all_small = neuro_re['MenteeID'][neuro_re.BigOrSmall==0]
sb = neuro_re['MenteeID'][neuro_re.Survival0==1]
sur_big = all_big[all_big.isin(sb)]
sur_small = all_small[all_small.isin(sb)]
print(len(sur_big)/len(all_big))
print(len(sur_small)/len(all_small))

0.5200282752120641
0.7670236411396242


导出数据 (export data)

In [86]:
neuro_re.to_csv('test0_0.25.csv',index=0)

批量输出 cem 匹配截断点 (export cutting points for coarsen matching)

In [87]:
cutpoint_list = ['First5YearPubsOfMentee','First5YearCitationOfMentee','First5YearCollaOfMentee',
                 'CollaPubsOfWithMentor',
                     'TotalPubsOfMentor','TotalCitationOfMentor','TotalCollaOfMentor',
                     'YearlyPubsOfMentor','YearlyCitationOfMentor','YearllyCollaOfMentor',
                 'PubsOfMentorInTraning']

In [88]:
CutPoint = pd.DataFrame()
for k in cutpoint_list:
#     tp = np.array(neuro_re[k])
    tp = np.array(neuro[k])
    # plt.hist(tp,range=[0,1000],bins=100)
    # print(a)
    print(k)
    qt = [0,0.3,0.6,1]
    cut = []
    for q in qt:
        cut.append(np.quantile(tp,q))
    CutPoint[k] = cut    
#     print(cut)

First5YearPubsOfMentee
[0, 3.0, 8.0, 134]
First5YearCitationOfMentee
[0, 56.0, 284.0, 232661]
First5YearCollaOfMentee
[0, 4.0, 11.0, 294]
CollaPubsOfWithMentor
[0, 0.0, 1.0, 77]
TotalPubsOfMentor
[0, 78.0, 237.0, 3290]
TotalCitationOfMentor
[0, 2058.0, 9921.0, 309472]
TotalCollaOfMentor
[0, 76.0, 250.0, 4303]
YearlyPubsOfMentor
[0.0, 1.8113207547169807, 4.531914893617022, 80.24390243902441]
YearlyCitationOfMentor
[0.0, 51.0, 184.82142857142856, 6393.69696969697]
YearllyCollaOfMentor
[0.0, 1.85, 4.760695652173912, 71.07843137254902]
PubsOfMentorInTraning
[0, 9.0, 32.0, 561]


In [2]:
#手动补充和修正0特多的变量的cutpoint (minor adjustment for cutting points)
cp = ['CollaPubsOfWithMentor']
for k in cp:
#     tp = np.array(neuro_re[k])
    tp = np.array(neuro[k])
    tp = tp[tp>0]
    # print(a)
#     print(k)
    qt = [0,0.5,1]
    cut = [0]
    for q in qt:
        cut.append(np.quantile(tp,q))
#     print(cut)
    CutPoint[k] = cut
CutPoint['CareerAgeOfMentorInTraining'] =  [0,15,25,50]

# CutPoint['TotalPubsOfMentor'] =  [0, 1000, 6000, 20854.0, 309472]
CutPoint['TotalCitationOfMentor'] =  [0, 100, 3000,309472]
# CutPoint['TotalCollaOfMentor'] =  [0, 50, 200, 526.0, 12568]
CutPoint['YearllyCollaOfMentor'] =  [0, 2, 8, 88]
# CutPoint['PubsOfMentorInTraning'] =  [0, 6,20,500]

导出数据 (export data)

In [92]:
CutPoint.to_csv('CutPoint0_0.25.csv',index=0)

### 生存下来的学生的数据 (data of survived mentees)

In [94]:
neuro_re1 = neuro[['MenteeID', 'MentorID','StopYear','BirthYearOfMentee','First5YearPubsOfMentee',
                  'First5YearCitationOfMentee','First5YearCollaOfMentee','CollaPubsOfWithMentor',
                  'BigOrSmall',
                  'TotalPubsOfMentor','TotalCitationOfMentor','TotalCollaOfMentor','CareerAgeOfMentorInTraining',
                  'YearlyPubsOfMentor','YearlyCitationOfMentor','YearllyCollaOfMentor','PubsOfMentorInTraning',
        
                  'Survival0','Survival1','YearlyCitationOfMentee','Top25',
                   'NumOfMenteesOfTheMenteeIn20']][(neuro.CareerAgeOfMentorInTraining<50)&(neuro.CareerAgeOfMentorInTraining>0)
                                             &(neuro.BigOrSmall!=-1)&(neuro.Survival0==1)]
    

In [95]:
neuro_re1.shape

(6003, 22)

In [96]:
CutPoint1 = pd.DataFrame()
for k in cutpoint_list:
    tp = np.array(neuro_re1[k])
    # plt.hist(tp,range=[0,1000],bins=100)
    # print(a)
#     print(k)
    qt = [0,0.3,0.6,1]
    cut = []
    for q in qt:
        cut.append(np.quantile(tp,q))
    CutPoint1[k] = cut    
#     print(cut)

First5YearPubsOfMentee
[0, 5.0, 10.0, 134]
First5YearCitationOfMentee
[0, 115.0, 353.0, 72767]
First5YearCollaOfMentee
[0, 6.0, 13.0, 170]
CollaPubsOfWithMentor
[0, 0.0, 2.0, 56]
TotalPubsOfMentor
[1, 106.0, 278.0, 2791]
TotalCitationOfMentor
[0, 3273.7999999999997, 11752.0, 283698]
TotalCollaOfMentor
[1, 104.0, 281.0, 4303]
YearlyPubsOfMentor
[0.04166666666666666, 2.2289408866995064, 5.063333333333333, 42.473684210526315]
YearlyCitationOfMentor
[0.0, 69.3793103448276, 221.73584905660374, 5959.452380952382]
YearllyCollaOfMentor
[0.017543859649122806, 2.248630136986301, 5.1124183006535935, 70.24]
PubsOfMentorInTraning
[0, 14.0, 37.0, 561]


In [98]:
#手动补充和修正0特多的变量的cutpoint (minor adjustment for cutting point)
cp = ['CollaPubsOfWithMentor']
for k in cp:
#     tp = np.array(neuro_re[k])
    tp = np.array(neuro[k])
    tp = tp[tp>0]
    # print(a)
#     print(k)
    qt = [0,0.5,1]
    cut = [0]
    for q in qt:
        cut.append(np.quantile(tp,q))
#     print(cut)
    CutPoint1[k] = cut
CutPoint1['CareerAgeOfMentorInTraining'] =  [0,15,25,50]

# CutPoint['TotalPubsOfMentor'] =  [0, 1000, 6000, 20854.0, 309472]
CutPoint1['TotalCitationOfMentor'] =  [0, 2000, 5000,309472]
# CutPoint['TotalCollaOfMentor'] =  [0, 50, 200, 526.0, 12568]
# CutPoint['YearllyCollaOfMentor'] =  [0, 2, 8, 88]
# CutPoint['PubsOfMentorInTraning'] =  [0, 6,19,300]

CollaPubsOfWithMentor
[0, 1, 4.0, 77]


导出数据 (export data)

In [99]:
neuro_re1.to_csv('test1_0.25.csv',index=0)

In [100]:
CutPoint1.to_csv('CutPoint1_0.25.csv',index=0)